# Use this notebook for developing mlframe prototype 2

## Object-oriented approach. Test 1.

In [246]:
import multiprocessing
from Queue import Queue, Empty
import subprocess
import os
import time
import re
import sys

In [247]:
def printObj(obj):
    for key,val in obj.__dict__.iteritems():
        print key,"=",val

In [273]:
# Class for executing commands in background processes.
# command_and_args should be an instance of class Command,
# but can also be a list or a string with the command and arguments separated with spaces.
# It can be used for executing remote processes in background. For that purpose exec_remote.sh should be used.
# Usage sample: 
# d = BashExecutor(command,hostname=hostname)
# d.start()
# exec_remote.sh must output subprocess exit code in the form:
# exitcode=N
# , where N is the number.
class BashExecutor(multiprocessing.Process):
    
    def __init__(self, command_and_args, debug=False, hostname=""):
        super(BashExecutor,self).__init__()
        self.debug = debug
        self.hostname = hostname
        if self.debug: print "command args:",command_and_args
        if type(command_and_args) is str:
            self.command = command_and_args.split(" ")
        else:
            self.command=command_and_args
        self.exitcode_pat = re.compile("^exitcode=(\d+)")
        manager = multiprocessing.Manager()
        d = manager.dict()
        self.d = d
        if isinstance(self.command,Command):
            self.d.command = self.command
        #if self.debug: print "self.command=",self.command
        #self.daemon = True
        
    # Poll exit code of self.proc and store it if not None.
    def poll(self):
        ec = self.proc.poll()
        if ec is not None:
            # Set exticode only if subprocess didn't set it __handleOutput__
            self.setExitCode(ec)
        return ec
    
    # Set given exit code to Command class object
    def setExitCode(self, ec):
        if self.debug: print "setting exit code to",ec
        self.d["exitcode"] = ec
        if isinstance(self.command, Command): 
            if self.d.command.exitcode is None:
                self.d.command.exitcode = ec
            if self.debug:
                print "Set exit code to",ec
                if ec is not None:
                    printObj(self.d.command)
            # Set Manager() dictionary "command" to Command object with set exitcode
            self.d["command"] = self.command
        
            
    
    # Copy process stdout to self.stdout queue line by line.
    def __handleOutput__(self, q):
        if self.debug: print self.name, "Handle output"
        proc = self.proc
        for line in iter(proc.stdout.readline, b''):
            m = self.exitcode_pat.match(line)
            if m is not None:
                if self.debug: print "=Exitcode match",m.group(0)
                self.setExitCode(m.group(0))
            q.put(line)
            #if self.debug:
            #    print self.hostname,line,
            sys.stdout.flush()
                    
    # Copy process stderr to self.stderr queue line by line.
    def __handleError__(self):
        if self.debug: print self.name, "Handle error"
        proc = self.proc
        for line in iter(proc.stderr.readline, b''):
            #self.stderr.put(line)
            if line is not None and len(line)>0:
                print "!",self.hostname,line,
        
    # Must be calles to start the process in background
    def run(self):
        if self.debug: 
            print "In {}. Calling {}".format(self.name,self.command)
            print "PID",os.getpid(),"name",multiprocessing.current_process().name
        if isinstance(self.command, Command):
            command = self.command.command.split(" ")
        else:
            command = self.command
        if self.debug: print "Executing",command
        proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=1, shell=False)
        self.proc = proc
        out_q = multiprocessing.Queue()
        p_out = multiprocessing.Process(target=self.__handleOutput__,args=(out_q,))
        p_out.start()
        p_err = multiprocessing.Process(target=self.__handleError__)
        p_err.start()
        if self.debug: print self.name, "process started"
        while self.poll() is None:
            try:
                line = out_q.get_nowait()                
                if self.debug:
                    print self.name,"stdout:",line
            except Empty:
                if self.debug: print "Queue empty"
            time.sleep(1)
            
        #self.__handleOutput__(proc)
        if self.debug:
            print "Exiting",multiprocessing.current_process().name, self.d["exitcode"]
            #print "self=",self
            #for key,val in self.__dict__.iteritems():
            #    print key,"=",val
        return
    
    def nameYourself(self):
        cp = multiprocessing.current_process()
        print "name",cp._name
        print "parent pid",cp._parent_pid
        print "id",cp._identity
    
    def getExitcode(self):
        print self.d
        if isinstance(self.command, Command):
            return self.d.command.exitcode
        return None
            

In [274]:
# Execute command on remote (or local) host in background, using exec_remote.sh and BashExecutor instance.
# Command can be any command or executable file name.
# Arguments must be in a string separated with spaces.
def RemoteExec(host, command_and_args, debug=False):    
    if debug: print "RemoteExec commands and args:",command_and_args
    CA = command_and_args.split(" ")
    if debug: print CA
    command = CA[0]
    args = CA[1:]
    if debug: print "Command:",command
    if debug: print "Args:",args
    package_directory = os.path.dirname(os.getcwd())
    scripts_location=os.path.realpath(os.path.join(package_directory,"mlframe","scripts"))
    exec_remote_script="exec_remote.sh"
    script_path = os.path.join(scripts_location,exec_remote_script)
    
    # Merge args into a string
    command_script_path = os.path.join(scripts_location, command)
    if debug: print "Test if script file exists in RemoteExec",command_script_path
    if os.path.isfile(command_script_path):
        command = command_script_path
    if debug: print "Command:",command
          
    if host == "" or host == "localhost" or host == "127.0.07":
        command_args_list = [command] + args
    else:
        command_args_list = [script_path] + [host] + [command] + args
    if debug: print "Calling BashExecutor with args",command_args_list
    d = BashExecutor(command_args_list, hostname=host, debug=debug)
    d.start()

    #for d in jobs:
    #    d.join()
    
    if debug: print "finished",host,command

In [275]:
# Class for storing command, it's stdout, stderr and exit code
class Command(object):
    
    def __init__(self, command):
        self.command = command
        self.stdout=""
        self.stderr=""
        self.exitcode=None
        

In [276]:
# Class for storing host-related data: hostname, access key and username.
# Stores commands (instances of Command class) executed on the host.
# Has methods for connecting to the host with ssh, connection test, executing commands.
class Host(object):
    
    commands = []
    
    def __init__(self, hostname, address = "localhost", user = "", key = "",debug=False):
        self.hostname = hostname
        self.address = address
        self.user = user
        self.key = key
        self.debug = debug
        ssh_command = "ssh "
        if key != "":
            ssh_command += "-i "+key+" "
        if user != "":
            ssh_command += user+"@"
        ssh_command += address
        self.ssh_command = ssh_command
        if debug:
            print "hostname,address,user,key:",self.hostname,self.address,self.user,self.key
            print "ssh command:",ssh_command
        
    def ping(self, N=5):
        d = BashExecutor("ping -c "+str(N)+" "+self.address, debug=self.debug)
        d.start()
        while d.getExitcode() is None:
            print d.getExitcode()
            time.sleep(3)
        if self.debug:
            print "Finished with exit code:"
        print d.getExitcode()
        
    
    def connect_test(self):
        options = " -o ConnectTimeout=5"
        command = self.ssh_command+options+ " hostname"        
        proc = subprocess.Popen(command.split(" "), 
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        for line in iter(proc.stdout.readline, b''):
            print line
        for line in iter(proc.stderr.readline, b''):
            print "err:",line
        

In [277]:
d = BashExecutor("ping -c 2 mouse.local", debug=True)
d.start()
print d

command args: ping -c 2 mouse.local
<BashExecutor(BashExecutor-65, started)>
In BashExecutor-65. Calling ['ping', '-c', '2', 'mouse.local']
PID 72108 name BashExecutor-65
Executing ['ping', '-c', '2', 'mouse.local']
BashExecutor-65 process started
Queue empty
BashExecutor-65 Handle error
BashExecutor-65 Handle output
BashExecutor-65 stdout: PING mouse.local (192.168.83.30): 56 data bytes

setting exit code to 0
Exiting BashExecutor-65 0


In [278]:
print d.getExitcode()

{'exitcode': 0}
None


In [279]:
comm = Command("ping -c 2 jetson.local")
d2 = BashExecutor(comm, debug=True)
d2.start()
print d2

command args: <__main__.Command object at 0x11091ca50>
<BashExecutor(BashExecutor-67, started)>
In BashExecutor-67. Calling <__main__.Command object at 0x11091ca50>
PID 72201 name BashExecutor-67
Executing ['ping', '-c', '2', 'jetson.local']
BashExecutor-67 process started
Queue empty
BashExecutor-67 Handle error
BashExecutor-67 Handle output
Queue empty
Queue empty
Queue empty
Queue empty
!  ping: cannot resolve jetson.local: Unknown host
setting exit code to 68
Set exit code to 68
exitcode = 68
command = ping -c 2 jetson.local
stderr = 
stdout = 
Exiting BashExecutor-67 68


In [284]:
print d2.getExitcode()
print d2.command.exitcode

{'command': <__main__.Command object at 0x11091cfd0>, 'exitcode': 68}
None
None


In [ ]:
key = "~/.ssh/id_rsa_com"
mouse = Host("mouse","mouse.local",user="peter",key=key,debug=True)

In [ ]:
mouse.ping(1)
# mouse.connect_test()

In [ ]:
muse = Host("muse","52.158.238.181", user="ubuntu", key=key,debug=False)

In [ ]:
muse.ping()
muse.connect_test()

In [ ]:
reedbush = Host("reedbush","reedbush.cc.u-tokyo.ac.jp",